## Importing Necessary Libraries

In [ ]:
import cv2 as cv
import os
import numpy as np

## Function to evaluate Frame Quality

Evaluate the quality of a frame based on edge density and contour count.

Parameters:\
frame (numpy.ndarray): The image frame to evaluate.\
edge_density_threshold (float): Minimum edge density ratio (edge pixels / total pixels).\
contour_count_threshold (int): Minimum number of contours required.

Returns:\
bool: True if the frame is of good quality, False otherwise.

In [ ]:
def evaluate_frame_quality(frame, edge_density_threshold=0.02, contour_count_threshold=100):
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY) #Converting to grayscale for further processing

    # Detect edges using Canny
    edges = cv.Canny(gray, 100, 200)
    edge_pixels = np.sum(edges > 0) #Number of pixels detected by canny edge detector
    total_pixels = gray.shape[0] * gray.shape[1] #Getting total by multiplying resolution
    edge_density = edge_pixels / total_pixels

    # Find contours
    contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    contour_count = len(contours) #No. of contours

    # Evaluate based on thresholds
    return edge_density >= edge_density_threshold and contour_count >= contour_count_threshold 
    '''if the edge density is greater than or equal to the default edge density (0.02) and the contour count
    is greater than or equal to the default contour threshold then it will return true which means the frame is of 
    good quality which we want'''

## Extracting and Filtering needed Frames 

Extracts frames from a video, filters them based on quality, and saves high-quality frames.

Parameters:\
video_path (str): Path to the input video file.\
output_folder (str): Folder to save filtered frames.\
frame_rate (int): Number of frames to extract per second.

In [ ]:
def extract_and_filter_frames(video_path, output_folder, frame_rate=2):
    if not os.path.exists(output_folder): # If output folder doesn't exist then make one
        os.makedirs(output_folder)

    # Open video file
    cap = cv.VideoCapture(video_path) 
    fps = int(cap.get(cv.CAP_PROP_FPS)) # Getting FPS of the video
    frame_count = 0
    success, frame = cap.read()

    while success:
        # Extract frames at the desired frame rate
        if frame_count % (fps // frame_rate) == 0:
            if evaluate_frame_quality(frame):  # Filter frame based on quality
                frame_name = f"{output_folder}/frame_{frame_count:04d}.jpg"
                cv.imwrite(frame_name, frame)  # Save high-quality frame

        success, frame = cap.read()
        frame_count += 1

    cap.release()
    print(f"High-quality frames saved to {output_folder}") # Confirmation that every frame saved successfully

## Final Implementation

In [ ]:
video_path = 'Video.mp4'  # Path to the video file
os.makedirs("filtered_frames",exist_ok=True)
output_folder = 'filtered_frames'  # Folder to save high-quality frames
frame_rate = 10  # Extract 10 frames per second

extract_and_filter_frames(video_path, output_folder, frame_rate) # Finally filtering and saving good frames